In [19]:
import random
from ipycanvas import Canvas, hold_canvas
import time

def populate(width, height): # Make a table (map) with random cells alive
    table = []
    choices = [0, 1]
    for i in range(height):
        temp = []
        for j in range(width):
            temp.append(random.choice(choices))
        table.append(temp)
    return table

def alive_next_generation(neighborhood):
    neighbors = sum(sum(row) for row in neighborhood) - neighborhood[1][1] # Count number of neighbors
    alive = neighborhood[1][1]
    if alive:
        if neighbors < 2 or neighbors > 3:
            return False
        else:
            return True
    else:
        if neighbors == 3:
            return True
        else:
            return False
    
def next_generation(world):
    new_world = [[0 for _ in range(len(world[0]))] for _ in range(len(world))]

    # Go through all cells
    for y in range(len(world)):
        for x in range(len(world[y])):
            # Make a neighborhood
            neighborhood = []
            for i in range(3):
                temp = []
                # Fix out of range error by going back to the start of the canvas
                yrange = y
                if (y-1)+i > len(world):
                        yrange -= len(world)
                for j in range(3):
                    xrange = x
                    if (x-1)+j > len(world[y]):
                        xrange -= len(world[y])
                    
                    temp.append(world[(yrange-2)+i][(xrange-2)+j]) # Add value to temp list for neighborhood
                neighborhood.append(temp)
            alive = alive_next_generation(neighborhood) # Check if alive based on neighborhood
            if alive:
                new_world[y][x] = 1
            else:
                new_world[y][x] = 0
    return new_world

width = 400
height = 400
cell_size = 40
fps = 5
runtime = 10
canvas = Canvas(width = width, height = height)
tick = 0

# world = populate(int(width/cell_size), int(height/cell_size))

world = [[0,0,0,0,1,0,0,0,0,0],
         [0,0,0,0,0,1,0,0,0,0],
         [0,0,0,1,1,1,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0]]

display(canvas)
while tick < runtime*fps:
    with hold_canvas(canvas):
        canvas.clear()
        for i in range(len(world)):
            for j in range(len(world[i])):
                posX = j*cell_size
                posY = i*cell_size
                if world[i][j] == 1:
                    canvas.fill_style = "black"
                    canvas.fill_rect(posX, posY, cell_size, cell_size)

        world = next_generation(world)
        tick += 1
        time.sleep(1/fps)

Canvas(height=400, width=400)